In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir('/content/drive/My Drive/')

Mounted at /content/drive


In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Load the CSV file
csv_path = '/content/drive/My Drive/ML2/ISIC_2019_Training_GroundTruth.csv'
df = pd.read_csv(csv_path)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
print(df)

              image  MEL   NV  BCC   AK  BKL   DF  VASC  SCC  UNK
0      ISIC_0000000  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
1      ISIC_0000001  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
2      ISIC_0000002  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
3      ISIC_0000003  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
4      ISIC_0000004  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
...             ...  ...  ...  ...  ...  ...  ...   ...  ...  ...
25326  ISIC_0073247  0.0  0.0  1.0  0.0  0.0  0.0   0.0  0.0  0.0
25327  ISIC_0073248  0.0  0.0  0.0  0.0  1.0  0.0   0.0  0.0  0.0
25328  ISIC_0073249  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
25329  ISIC_0073251  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
25330  ISIC_0073254  0.0  0.0  0.0  0.0  1.0  0.0   0.0  0.0  0.0

[25331 rows x 10 columns]


In [ ]:
from torchvision import transforms
class SkinLesionDataset(torch.utils.data.Dataset):
    def __init__(self, df, image_dir, transform=None, target_size=(384, 384)):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform
        self.target_size = target_size

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.df.iloc[idx, 0] + '.jpg')
        image = Image.open(img_name)
        resize_transform = transforms.Resize(self.target_size, interpolation=Image.BILINEAR)
        image = resize_transform(image)
        label = torch.tensor(self.df.iloc[idx, 1:].values.astype('float32'))

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# Define transformations and augmentation for different classes
transform_augment_scc_vasc_df_ak = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_augment_bkl_mel = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    #transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_augment_nv = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


'\ntransform_train = transforms.Compose([\n    transforms.Resize((224, 224)),\n    transforms.RandomHorizontalFlip(),\n    transforms.RandomVerticalFlip(),\n    transforms.RandomRotation(20),\n    transforms.ToTensor(),\n    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])\n])\n\ntransform_val_test = transforms.Compose([\n    transforms.Resize((224, 224)),\n    transforms.ToTensor(),\n    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])\n])\n'

In [ ]:
# Create dataset
image_dir = '/content/drive/My Drive/ML2/ISIC_2019_Training_Input/ISIC_2019_Training_Input'
dataset = SkinLesionDataset(df, image_dir, transform=None)

In [ ]:
# Split the dataset into training, validation, and testing sets
train_val_indices, test_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42, stratify=df.iloc[:, 1:])
train_indices, val_indices = train_test_split(train_val_indices, test_size=0.2, random_state=42, stratify=df.iloc[train_val_indices, 1:])

train_set = torch.utils.data.Subset(dataset, train_indices)
val_set = torch.utils.data.Subset(dataset, val_indices)
test_set = torch.utils.data.Subset(dataset, test_indices)

# Create data loaders
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

In [ ]:
print(len(train_set))
print(len(val_set))
print(len(test_set))

print(len(train_loader))
print(len(val_loader))
print(len(test_loader))

16211
4053
5067
507
127
159


In [ ]:
def custom_collate_fn(batch):
  images, labels = zip(*batch)
  augmented_images = []
  for i in range(len(images)):
    label = labels[i]
    if label[6] == 1 or label[7] == 1 or label[5] == 1 or label[3] == 1:
      augmented_images.append(transform_augment_scc_vasc_df_ak(images[i]))
    elif label[4] == 1 or label[0] == 1:
      augmented_images.append(transform_augment_bkl_mel(images[i]))
    else:
      augmented_images.append(transform_augment_nv(images[i]))
  augmented_images_resized = [transforms.Resize((384, 384))(img) for img in augmented_images]
  augmented_images = torch.stack(augmented_images)
  labels_tensor = torch.stack(labels)
  return augmented_images, labels_tensor

train_loader.collate_fn = custom_collate_fn

In [ ]:
def custom_val_collate_fn(batch):
    images, labels = zip(*batch)
    tensor_images = [transforms.ToTensor()(img) for img in images]
    tensor_labels = torch.stack(labels)
    return torch.stack(tensor_images), tensor_labels

val_loader.collate_fn = custom_val_collate_fn

In [ ]:
def custom_test_collate_fn(batch):
    images, labels = zip(*batch)
    tensor_images = [transforms.ToTensor()(img) for img in images]
    tensor_labels = torch.stack(labels)
    return torch.stack(tensor_images), tensor_labels

test_loader.collate_fn = custom_test_collate_fn

In [ ]:
model = torchvision.models.efficientnet_v2_s(pretrained=True)

num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 9)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 125MB/s]


In [ ]:
from torchsummary import summary
summary(model, (3, 384, 384))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 192, 192]             648
       BatchNorm2d-2         [-1, 24, 192, 192]              48
              SiLU-3         [-1, 24, 192, 192]               0
            Conv2d-4         [-1, 24, 192, 192]           5,184
       BatchNorm2d-5         [-1, 24, 192, 192]              48
              SiLU-6         [-1, 24, 192, 192]               0
   StochasticDepth-7         [-1, 24, 192, 192]               0
       FusedMBConv-8         [-1, 24, 192, 192]               0
            Conv2d-9         [-1, 24, 192, 192]           5,184
      BatchNorm2d-10         [-1, 24, 192, 192]              48
             SiLU-11         [-1, 24, 192, 192]               0
  StochasticDepth-12         [-1, 24, 192, 192]               0
      FusedMBConv-13         [-1, 24, 192, 192]               0
           Conv2d-14           [-1, 96,

In [ ]:
best_accuracy = 0.0
patience = 3
epochs_since_improvement = 0

model.to(device)

# Train the model
num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        print(f"Epoch {epoch}, Batch {batch_idx}")
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.argmax(dim=1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels.argmax(dim=1)).sum().item()

    train_accuracy = correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Training Accuracy: {100*train_accuracy}%')
    scheduler.step()

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
      for val_inputs, val_labels in val_loader:
        val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
        outputs = model(val_inputs)
        loss = criterion(outputs, val_labels.argmax(dim=1))
        val_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        val_total += labels.size(0)
        val_correct += (predicted == val_labels.argmax()).sum().item()

    val_accuracy = val_correct / val_total
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss/len(val_loader)}, Validation Accuracy: {100*val_accuracy}%')

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        epochs_since_improvement = 0
        if not os.path.exists('checkpoints'):
            os.makedirs('checkpoints')
        torch.save(model.state_dict(), f'checkpoints/best_model.pth')
    else:
        epochs_since_improvement += 1
        if epochs_since_improvement >= patience:
            print(f'No improvement in validation accuracy for {patience} epochs. Loading best model.')
            model.load_state_dict(torch.load('checkpoints/best_model.pth'))
            break

Epoch 0, Batch 0
Epoch 0, Batch 1
Epoch 0, Batch 2
Epoch 0, Batch 3
Epoch 0, Batch 4
Epoch 0, Batch 5
Epoch 0, Batch 6
Epoch 0, Batch 7
Epoch 0, Batch 8
Epoch 0, Batch 9
Epoch 0, Batch 10
Epoch 0, Batch 11
Epoch 0, Batch 12
Epoch 0, Batch 13
Epoch 0, Batch 14
Epoch 0, Batch 15
Epoch 0, Batch 16
Epoch 0, Batch 17
Epoch 0, Batch 18
Epoch 0, Batch 19
Epoch 0, Batch 20
Epoch 0, Batch 21
Epoch 0, Batch 22
Epoch 0, Batch 23
Epoch 0, Batch 24
Epoch 0, Batch 25
Epoch 0, Batch 26
Epoch 0, Batch 27
Epoch 0, Batch 28
Epoch 0, Batch 29
Epoch 0, Batch 30
Epoch 0, Batch 31
Epoch 0, Batch 32
Epoch 0, Batch 33
Epoch 0, Batch 34
Epoch 0, Batch 35
Epoch 0, Batch 36
Epoch 0, Batch 37
Epoch 0, Batch 38
Epoch 0, Batch 39
Epoch 0, Batch 40
Epoch 0, Batch 41
Epoch 0, Batch 42
Epoch 0, Batch 43
Epoch 0, Batch 44
Epoch 0, Batch 45
Epoch 0, Batch 46
Epoch 0, Batch 47
Epoch 0, Batch 48
Epoch 0, Batch 49
Epoch 0, Batch 50
Epoch 0, Batch 51
Epoch 0, Batch 52
Epoch 0, Batch 53
Epoch 0, Batch 54
Epoch 0, Batch 55
Ep

In [ ]:
# Evaluate the model on the test set
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
  for test_inputs, test_labels in test_loader:
    test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
    outputs = model(test_inputs)
    loss = criterion(outputs, test_labels.argmax(dim=1))
    test_loss += loss.item()
    _, predicted = torch.max(outputs, 1)
    test_total += labels.size(0)
    test_correct += (predicted == test_labels.argmax()).sum().item()


test_accuracy = test_correct / test_total
print(f'Test Accuracy: {100*test_accuracy}%')

Test Accuracy: 76.00132406487919%


"\nwith torch.no_grad():\n  for test_inputs, test_labels in test_set:\n    test_inputs = transform(test_inputs).unsq ueeze(0)\n    test_labels = torch.tensor(test_labels).unsqueeze(0)\n    test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)\n    #test_inputs = torch.stack([transforms.ToTensor()(img) for img in test_inputs])\n    #test_labels = torch.tensor(test_labels)\n    outputs = model(test_inputs)\n    loss = criterion(outputs, test_labels.argmax(dim=1))\n    test_loss += loss.item()\n    _, predicted = torch.max(outputs, 1)\n    test_total += labels.size(0)\n    test_correct += (predicted == test_labels.argmax()).sum().item()\n    #for inputs, labels in test_loader:\n#    for test_inputs, test_labels in test_loader:\n#      test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)\n#      for test_input_idx in range(len(test_inputs)):\n#        #outputs = model(inputs)\n#        outputs = model(inputs[test_input_idx].unsqueeze(0))\n#        _